In [1]:
import os
import pandas as pd
import numpy as np
#import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV
from sklearn import datasets
from collections import Counter

from joblib import dump
from sklearn.externals import joblib

#from keras.models import model_from_json

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score


/home/estudiante/fcrv/jupyter_notebooks/data_analisis_env_py3/lib/python3.5/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


# Se importa el Dataset con informacion de nubes
Como se ha descrito, las imagenes descargadas de los portales de imagenes satelitales presentan información en multiples bandas. Es con esta información que los algoritmos de clasificación pueden emitir un juicio para determinar a que tipo de información puede corresponder un pixel: bosque, no busque,nubes,agua, agua_bosque,ninguno.
En este escenario se plantea probar el clasificador Naive Bayes con información de imagenes que contengan nubes y agregar una nueva clase: Nubes. 
Este escenario se plantea en respuesta a que la clasificación de imagenes satelitales cuyas nubes fueron removidas resultaban un problema facil de clasificar para este tipo de modelo. 

In [2]:
path = './satellite_dataset-con-nubes.csv'

satellite = pd.read_csv(path)
satellite.head(100)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,cloud
0,208.0,168.0,88.0,86.0,91.0,72.0,1,0,0,0,0
1,170.0,118.0,40.0,27.0,61.0,53.0,1,0,0,0,0
2,357.0,247.0,143.0,119.0,116.0,90.0,1,0,0,0,0
3,263.0,180.0,87.0,59.0,64.0,45.0,1,0,0,0,0
4,3359.0,3167.0,3127.0,3150.0,2731.0,2703.0,0,0,0,0,1
5,904.0,707.0,549.0,450.0,316.0,222.0,1,0,0,0,0
6,224.0,148.0,61.0,41.0,66.0,54.0,1,0,0,0,0
7,439.0,350.0,214.0,160.0,140.0,104.0,1,0,0,0,0
8,1635.0,2032.0,1949.0,4395.0,3198.0,2215.0,0,0,0,1,0
9,5840.0,6109.0,6246.0,7067.0,5533.0,4197.0,0,0,0,0,1


# Filtro de valores negativos
Se realiza una filtro de las columnas que en alguna banda contengan un valor negativo porque el clasificador multinomial Naive bayes no funciona bien con esta información.

In [3]:
satellite = satellite[satellite["blue"] > 0]
satellite = satellite[satellite["red"] > 0]
satellite = satellite[satellite["green"] > 0]
satellite = satellite[satellite["nir"] > 0]
satellite = satellite[satellite["swir1"] > 0]
satellite = satellite[satellite["swir2"] > 0]

In [4]:
#satellite['wofs'].value_counts()
#satellite['bosque'].value_counts()
#satellite['wofs_bosque'].value_counts()
#satellite['cloud'].value_counts()
#satellite['ninguno'].value_counts()

# Se reemplaza los valores en columnas de clases
Como parte de preparación de los datos para crear el modelo se deben hacer algunos ajustes a la forma en que el dataset muestra la infiromación.Como se puede observar el dataset, las columnas de las clases se encuentran categorizadas por un valor binario. Lo que se realiza a continuación es reemplazar estos valores binarios por NaN en caso de ser "Cero" y el nombre de la clase en caso de ser "Uno".

In [5]:
# Reemplazar el valor de columna Wofs por NaN(0) y Wofs(1)
satellite['wofs'] = satellite['wofs'].replace(0,np.nan)
satellite['wofs'] = satellite['wofs'].replace(1,'wofs')


In [6]:
# Reemplazar el valor de columna bosque por NaN(0) y bosque(1)
satellite['bosque'] = satellite['bosque'].replace(0,np.nan)
satellite['bosque'] = satellite['bosque'].replace(1,'bosque')

In [7]:
# Reemplazar el valor de columna wofs_bosque por NaN(0) y wofs_bosque(1)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(0,np.nan)
satellite['wofs_bosque'] = satellite['wofs_bosque'].replace(1,'wofs_bosque')

In [8]:
# Reemplazar el valor de columna ninguno por NaN(0) y ninguno(1)
satellite['ninguno'] = satellite['ninguno'].replace(0,np.nan)
satellite['ninguno'] = satellite['ninguno'].replace(1,'ninguno')

In [9]:
# Reemplazar el valor de columna ninguno por NaN(0) y cloud(1)
satellite['cloud'] = satellite['cloud'].replace(0,np.nan)
satellite['cloud'] = satellite['cloud'].replace(1,'cloud')

# Nueva columna de clasificación
Como ultima parte para la preparación de los datos debemos tener la clasificación en una sola columna. Procedemos a crearla con el nombre "label" y alli asignaremos el valor categorizado de cada conjunto de atributos.

In [10]:
#Se crea una nueva columna para almacenar el valor de etiqueta
satellite['label']=np.nan

#Se indica en la columna 'label', la clasificacion de la columna 'wofs'. Se esribe "Wofs" en columna 'label' si en  
#la columna 'wofs' esta este valor
for label in [col for col in satellite.columns if 'wofs' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'bosque'. Se esribe "bosque" en columna 'label' si en  
#la columna 'bosque' esta este valor
for label in [col for col in satellite.columns if 'bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'wofs_bosque'. Se esribe "wofs_bosque" en columna 'label' si en  
#la columna 'wofs_bosque' esta este valor
for label in [col for col in satellite.columns if 'wofs_bosque' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'ninguno'. Se esribe "ninguno" en columna 'label' si en  
#la columna 'ninguno' esta este valor
for label in [col for col in satellite.columns if 'ninguno' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
    
#Se indica en la columna 'label', la clasificacion de la columna 'cloud'. Se esribe "cloud" en columna 'label' si en  
#la columna 'cloud' esta este valor
for label in [col for col in satellite.columns if 'cloud' in col]:
    satellite['label'].fillna(satellite[label],inplace=True)
satellite.head(20)

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno,cloud,label
0,208.0,168.0,88.0,86.0,91.0,72.0,wofs,NaN,NaN,NaN,NaN,wofs
1,170.0,118.0,40.0,27.0,61.0,53.0,wofs,NaN,NaN,NaN,NaN,wofs
2,357.0,247.0,143.0,119.0,116.0,90.0,wofs,NaN,NaN,NaN,NaN,wofs
3,263.0,180.0,87.0,59.0,64.0,45.0,wofs,NaN,NaN,NaN,NaN,wofs
4,3359.0,3167.0,3127.0,3150.0,2731.0,2703.0,NaN,NaN,NaN,NaN,cloud,cloud
5,904.0,707.0,549.0,450.0,316.0,222.0,wofs,NaN,NaN,NaN,NaN,wofs
6,224.0,148.0,61.0,41.0,66.0,54.0,wofs,NaN,NaN,NaN,NaN,wofs
7,439.0,350.0,214.0,160.0,140.0,104.0,wofs,NaN,NaN,NaN,NaN,wofs
8,1635.0,2032.0,1949.0,4395.0,3198.0,2215.0,NaN,NaN,NaN,ninguno,NaN,ninguno
9,5840.0,6109.0,6246.0,7067.0,5533.0,4197.0,NaN,NaN,NaN,NaN,cloud,cloud


wofs
cloud
ninguno
boque
wofs_bosque

In [11]:
# Se crea el conjunto de clasificacion y el de entrenamiento
Y = satellite["label"]
X = satellite.drop(columns=["label","wofs","bosque","wofs_bosque","cloud","ninguno"], axis=1)
Y.value_counts()

cloud          395787
wofs           340403
bosque         215233
ninguno         45327
wofs_bosque       181
Name: label, dtype: int64

# Se crean los conjuntos de entrenamiento y de prueba
Para hacer esto se deben dividir los datos en 4 conjuntos de datos de la siguiente manera: Conjuntos de entrenamiento(X_train, y_train) y Conjunto de test(X_test, y_test)

In [12]:
# Se dividen los datos en los conjuntos de Train y test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    Y,
    test_size=0.3,
    random_state=0
)

In [13]:
X_test

,blue,green,red,nir,swir1,swir2
320143,221.0,161.0,79.0,65.0,89.0,76.0
39220,5598.0,5827.0,5909.0,6855.0,5425.0,4121.0
985443,3079.0,3231.0,3193.0,4799.0,3636.0,2970.0
681309,4259.0,4450.0,4402.0,5495.0,4105.0,2919.0
700134,219.0,384.0,239.0,3011.0,1514.0,601.0
514756,196.0,149.0,68.0,54.0,80.0,66.0
944640,212.0,383.0,221.0,3066.0,1278.0,502.0
913436,708.0,671.0,512.0,2502.0,1160.0,550.0
927309,3070.0,3102.0,3082.0,4944.0,3876.0,2966.0
516675,237.0,334.0,174.0,1686.0,642.0,235.0


In [14]:
y_test

320143       wofs
39220       cloud
985443      cloud
681309      cloud
700134     bosque
514756       wofs
944640     bosque
913436    ninguno
927309      cloud
516675     bosque
375193       wofs
549948      cloud
214648       wofs
146169       wofs
416196      cloud
148581     bosque
874345       wofs
840112     bosque
621568     bosque
989775      cloud
996506      cloud
825551      cloud
780541       wofs
512451      cloud
225029       wofs
152186     bosque
273858      cloud
895864      cloud
677169      cloud
748018       wofs
           ...   
821481       wofs
257829      cloud
937050      cloud
624765       wofs
914965    ninguno
229550     bosque
942430      cloud
815307       wofs
91960        wofs
709128       wofs
311443      cloud
772444       wofs
950020     bosque
572626     bosque
42690      bosque
260699      cloud
457352     bosque
503600       wofs
252506      cloud
445252     bosque
269617      cloud
647683     bosque
605316       wofs
641103     bosque
806561    

# Naive Bayes para multiples capas
La clasificación multicapa del algoritmo Naive Bayes es conocida como "Clasificación Naive Bayes Multinomial". Como se puede apreciar la clasificación de nuestro conjunto de caracteristicas puede tener 4 valores: wofs, bosque, wofs_bosque y ninguno. 
En esta parte de creación del modelo se evaluaran dos variaciones del clasificador Naive Bayes: Gausiano y multinomial. En el analisis de resultados se contrastaran los resultados obtenidos para cada uno de ellos.

### Multinomial

In [15]:
#Create a Gaussian Classifier
gnb = MultinomialNB()

In [16]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [18]:
Counter(y_pred)

Counter({'bosque': 60048,
         'cloud': 109882,
         'ninguno': 27271,
         'wofs': 99543,
         'wofs_bosque': 2336})

In [19]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 59236,      2,   5475,      2,     10],
       [     7, 105994,  10958,    105,   1701],
       [   805,   1606,  10799,    102,    372],
       [     0,   2267,     24,  99333,    226],
       [     0,     13,     15,      1,     27]])

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.92      0.95     64725
       cloud       0.96      0.89      0.93    118765
     ninguno       0.40      0.79      0.53     13684
        wofs       1.00      0.98      0.99    101850
 wofs_bosque       0.01      0.48      0.02        56

    accuracy                           0.92    299080
   macro avg       0.67      0.81      0.68    299080
weighted avg       0.95      0.92      0.93    299080



In [21]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9207870803798315


In [23]:
joblib.dump(gnb, 'model_nb_multinomial.joblib') 

['model_nb_multinomial.joblib']

### Gaussiano

In [15]:
#Create a Gaussian Classifier
gnb = GaussianNB()

In [16]:
#Train the model using the training sets
gnb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [17]:
#Predict the response for test dataset
y_pred = gnb.predict(X_test)

In [18]:
Counter(y_pred)

Counter({'bosque': 60048,
         'cloud': 109882,
         'ninguno': 27271,
         'wofs': 99543,
         'wofs_bosque': 2336})

In [19]:
#wofs
#cloud
#ninguno
#bosque
#wofs_bosque
confusion_matrix(y_test, y_pred)

array([[ 59236,      2,   5475,      2,     10],
       [     7, 105994,  10958,    105,   1701],
       [   805,   1606,  10799,    102,    372],
       [     0,   2267,     24,  99333,    226],
       [     0,     13,     15,      1,     27]])

In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

      bosque       0.99      0.92      0.95     64725
       cloud       0.96      0.89      0.93    118765
     ninguno       0.40      0.79      0.53     13684
        wofs       1.00      0.98      0.99    101850
 wofs_bosque       0.01      0.48      0.02        56

    accuracy                           0.92    299080
   macro avg       0.67      0.81      0.68    299080
weighted avg       0.95      0.92      0.93    299080



In [21]:
# Model Accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9207870803798315


In [23]:
joblib.dump(gnb, 'model_nb_gaussiano.joblib') 

['model_nb_multinomial.joblib']

## Conclusiones

El rendmiento obtenido con el modelo Multinomial llega a ser del 79.73% y el rendimiento con el modelo Gausiano alcanza el 87.11%. Por lo que logramos determinar que en este escenario funciona mejor el modelo Gassiano que el Multinomial por aproimadamente 8 puntos de diferencia. Revisando la matriz de confusión logramos evidenciar que el modelo Multinomial confunde mucho de los elementos de la clase 3(Ninguno) siendo esto logico porque los valores de caracteristicas de entrada


**Multinomial**

Accuracy: 0.7973101132317454

array([[ 28721,      1,    320,      2,    551],
       [    43, 120350,  39851,    240,   1295],
       [  1895,   8902,   4897,   1003,   1646],
       [     0,   3826,    175,  82221,    278],
       [     0,     12,      7,      9,     50]], dtype=int64)
       
**Gaussiano**

Accuracy: 0.8711520612902681

array([[ 27133,     74,   1321,    349,    718],
       [    68, 134215,  27495,      1,      0],
       [  2570,   1176,  13427,   1065,    105],
       [    95,      0,   2685,  83335,    385],
       [    24,      0,      6,     40,      8]], dtype=int64)

# Referencias
https://www.datacamp.com/community/tutorials/naive-bayes-scikit-learn

https://datascience.stackexchange.com/questions/40345/how-to-convert-multiple-columns-into-single-columns-in-pandas

https://stackoverflow.com/questions/13295735/how-can-i-replace-all-the-nan-values-with-zeros-in-a-column-of-a-pandas-datafra